## 迭代器

可以直接作用于for循环的对象统称为可迭代对象：`Iterable`

一类是集合数据类型，如list、tuple、dict、set、str等；

一类是generator，包括生成器和带yield的generator function。

可以使用`isinstance()`判断一个对象是否是`Iterable`对象：

In [2]:
from collections.abc import Iterable
isinstance([], Iterable)
isinstance((x for x in range(10)), Iterable)

True

生成器不但可以作用于for循环，还可以被`next()`函数不断调用并返回下一个值，直到最后抛出`StopIteration`错误表示无法继续返回下一个值了。

可以被`next()`函数调用并不断返回下一个值的对象称为迭代器：`Iterator`

可以使用`isinstance()`判断一个对象是否是`Iterator`对象：

In [6]:
from collections.abc import Iterator
isinstance((x for x in range(10)), Iterator)
isinstance([], Iterator)

False

生成器都是`Iterator`对象，但list、dict、str虽然是`Iterable`，却不是`Iterator`。

把list、dict、str等Iterable变成Iterator可以使用`iter()`函数

In [7]:
isinstance(iter('abc'), Iterator)

True

`Iterator`甚至可以表示一个无限大的数据流，例如全体自然数。而使用list是永远不可能存储全体自然数的。

### 小结
凡是可作用于`for`循环的对象都是`Iterable`类型；

凡是可作用于`next()`函数的对象都是`Iterator`类型，它们表示一个惰性计算的序列；

集合数据类型如list、dict、str等是Iterable但不是Iterator，可以通过`ter()`函数获得一个Iterator对象。

Python的for循环本质上就是通过不断调用`next()`函数实现

## 高阶函数

### yield
 - yield在函数中的功能类似于return，不同的是yield每次返回结果之后函数并没有退出，而是 每次遇到yield关键字后返回相应结果，并保留函数当前的运行状态，等待下一次的调用

 - 如果 一个函数需要多次循环执行一个动作，并且每次执行的结果都是需要的，这种场景很适合使用yield实现。

包含yield的函数成为一个生成器，生成器同时也是一个迭代器，支持通过next方法获取下一个值



In [23]:
def foo():
    while True:
        res = yield 4
        print(res)
        
g = foo()
print(next(g))
print('*'*20)
print(next(g))

4
********************
None
4
********************
None
4


1.程序开始执行以后，因为foo函数中有yield关键字，所以foo函数并不会真的执行，而是先得到一个生成器g(相当于一个对象)

2.直到调用next方法，foo函数正式开始执行，先执行foo函数中的print方法，然后进入while循环

3.程序遇到yield关键字，然后把yield想想成return,return了一个4之后，程序停止，并没有执行赋值给res操作，此时next(g)语句执行完成，所以输出的前两行（第一个是while上面的print的结果,第二个是return出的结果）是执行print(next(g))的结果，

4.程序执行print("*"*20)，输出20个*

5.又开始执行下面的print(next(g)),这个时候和上面那个差不多，不过不同的是，这个时候是从刚才那个next程序停止的地方开始执行的，也就是要执行res的赋值操作，这时候要注意，这个时候赋值操作的右边是没有值的（因为刚才那个是return出去了，并没有给赋值操作的左边传参数），所以这个时候res赋值是None,所以接着下面的输出就是res:None,

6.程序会继续在while里执行，又一次碰到yield,这个时候同样return 出4，然后程序停止，print函数输出的4就是这次return出的4.

 

yield和return的关系和区别:带yield的函数是一个生成器，而不是一个函数了，这个生成器有一个函数就是next函数，next就相当于“下一步”生成哪个数，这一次的next开始的地方是接着上一次的next停止的地方执行的，所以调用next的时候，生成器并不会从foo函数的开始执行，只是接着上一步停止的地方开始，然后遇到yield后，return出要生成的数，此步就结束。


### map/reduce

#### map
`map()`函数接收两个参数，一个是函数，一个是`Iterable`，`map`将传入的函数依次作用到序列的每个元素，并把结果作为新的`Iterator`返回。

比如我们有一个函数 f(x) = x^2, 现在需要将这个函数作用在一个list=[1,2,3,4,5,6,7], 可以使用`map`实现:

In [9]:
def f(x):
    return x*x
r = map(f, [1,2,3,4,5,6,7])
print(list(r))

[1, 4, 9, 16, 25, 36, 49]


`map`传入的地哦一个参数是f， 即函数本身。由于结果r 是一个`Iterator`，`Iterator` 是惰性序列。因此通过`list()`函数可以让他把整个序列都计算出来并返回一个list.

In [11]:
L = []
for n in range(1,10):
    L.append(f(n))
print(L)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


把list所有数字转化为字符串

In [ ]:
list(map(str, [1,2,23,4,4,4,5]))

#### Reduce

`reduce`把一个函数作用在一个序列`[x1, x2, x3, ...]`上，这个函数必须接收两个参数，`reduce`把结果继续和序列的下一个元素做累积计算，其效果就是：
```
reduce(f,[x1,x2,x3,x4]) = f(f(f(x1,x2,),x3), x4)
```
以下实例是对一个序列求和

In [13]:
from functools import reduce
def add(x, y):
    return x+y

output = reduce(add, [1,3,5,7])
print(output)

16


把字符串转化为数字：

In [17]:
from functools import reduce

DIGITS = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}

def str2int(s):
    def fn(x,y):
        return x*10 + y
    def char2num(s):
        return DIGITS[s]
    return reduce(fn,map(char2num, s))

### filter

和`map()`类似，`filter()`也接收一个函数和一个序列。和`map()`不同的是，`filter()`把传入的函数依次作用于每个元素，然后根据返回值是True还是False决定保留还是丢弃该元素。

In [19]:
def is_odd(x):
    return x%2 == 1

filtered = filter(is_odd, [1,2,3,4,5,6])
print(list(filtered))

[1, 3, 5]


`filter()`函数返回的是一个`Iterator`，也就是一个惰性序列，所以要强迫`filter()`完成计算结果，需要用list()函数获得所有结果并返回list。

#### 用filter求素数

构造一个从3开始的序列

In [32]:
def _odd_iter():
    n  = 1
    while True:
        n  = n + 2
        yield n

这是一个生成器，并且是一个无限序列， 然后定义一个筛选函数：

In [36]:
def _not_divisible(n):
    return lambda x: x%n >0

True

最后定义一个生成器，不断返回下一个素数：

In [28]:
def primes():
    yield 2
    it = _odd_iter()
    while True:
        n = next(it)
        yield n
        it = filter(_not_divisible(n), it)

这个生成器先返回第一个素数2，然后，利用`filter()`不断产生筛选后的新的序列。

由于`primes()`也是一个无限序列，所以调用时需要设置一个退出循环的条件：

In [33]:
# 打印1000以内的素数:
for n in primes():
    if n < 1000:
        print(n)
    else:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101
103
107
109
113
127
131
137
139
149
151
157
163
167
173
179
181
191
193
197
199
211
223
227
229
233
239
241
251
257
263
269
271
277
281
283
293
307
311
313
317
331
337
347
349
353
359
367
373
379
383
389
397
401
409
419
421
431
433
439
443
449
457
461
463
467
479
487
491
499
503
509
521
523
541
547
557
563
569
571
577
587
593
599
601
607
613
617
619
631
641
643
647
653
659
661
673
677
683
691
701
709
719
727
733
739
743
751
757
761
769
773
787
797
809
811
821
823
827
829
839
853
857
859
863
877
881
883
887
907
911
919
929
937
941
947
953
967
971
977
983
991
997


### 返回函数

#### 函数作为返回值

不需要立刻求和，而是在后面的代码中，根据需要再计算 ---- 返回求和的函数：



In [38]:
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax = ax + n
        return ax
    return sum

当我们调用lazy_sum()时，返回的并不是求和结果，而是求和函数

In [39]:
f = lazy_sum(1,3,5,7,9)
print(f)

<function lazy_sum.<locals>.sum at 0x7f1c92e37550>


In [40]:
f()

25

#### 闭包

注意到返回的函数在其定义内部引用了局部变量args，所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用

另一个需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了f()才执行。例子：

In [44]:
def count():
    fs = []
    for i in range(1, 4):
        def f():
            return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()
print(f1(), f2(), f3())

9 9 9


在上面的例子中，每次循环，都创建了一个新的函数，然后，把创建的3个函数都返回了。

很可能错误的以为调用f1()，f2()和f3()结果应该是`1，4，9`，但实际结果是

全部都是`9`！原因就在于返回的函数引用了变量`i`，但它并非立刻执行。等到3个函数都返回时，它们所引用的变量i已经变成了3，因此最终结果为`9`。

所以 <b>返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。</b>

如果一定要引用循环变量怎么办？

方法:再创建一个函数，用该函数的参数绑定循环变量当前的值，无论该循环变量后续如何更改，已绑定到函数参数的值不变

In [46]:
def count():
    def f(j):
        def g():
            return j*j
        return g
    fs = []
    for i in range(1,4):
        fs.append(f(i))
    return fs

f1, f2, f3 = count()
print(f1(), f2(), f3())

1 4 9


### 装饰器
由于函数也是一个对象，而且函数对象可以被赋值给变量，所以，通过变量也能调用该函数

函数对象有一个__name__属性，可以拿到函数的名字：

In [51]:
def now():
    print('function')
    
f = now
f()
print(now.__name__)

function
now


现在，假设我们要增强`now()`函数的功能

比如，在函数调用前后自动打印日志，但又不希望修改`now()`函数的定义

使用“装饰器”（Decorator），在代码运行期间动态增加功能

<b>本质上，decorator就是一个返回函数的高阶函数。</b>

所以，我们要定义一个能打印日志的decorator，可以定义如下：

In [52]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s()' %func.__name__)
        return func(*args, **kw)
    return wrapper

上面的`log`，因为它是一个`decorator`

所以接受一个函数作为参数，并返回一个函数。借助Python的@语法，把decorator置于函数的定义处：

In [54]:
@log
def now():
    print('function')
now()

call now()
function


把@log放到now()函数的定义处，相当于执行了语句：

In [55]:
now = log(now)

由于`log()`是一个`decorator`，返回一个函数，所以，原来的`now()`函数仍然存在，只是现在同名的`now`变量指向了新的函数，于是调用`now()`将执行新函数，即在`log()`函数中返回的`wrapper()`函数

`wrapper()`函数的参数定义是`(*args, **kw)`，因此，`wrapper(`)函数可以接受任意参数的调用。在`wrapper()`函数内，首先打印日志，再紧接着调用原始函数。

如果decorator本身需要传入参数，那就需要编写一个返回`decorator`的高阶函数，写出来会更复杂。比如，要自定义log的文本：

In [57]:
def log(text):
    def decorator(func):
        def wrapper(*args, **kw):
            print('%s %s()' % (text, func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator

@log('execute')
def now():
    print('function')
now()


execute now()
function


不需要编写`wrapper.__name__ = func.__name__`这样的代码，Python内置的`functools.wraps`就是干这个事的，所以，一个完整的decorator的写法如下

In [58]:
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s()' % func.__name__)
        return func(*args, **kw)
    return wrapper

#### 实例

（1）最简单的装饰器，实现日志输出功能

In [59]:
# 构建装饰器
def logging(func):
    @functools.wraps(func)
    def decorator():
        print("%s called" % func.__name__)
        result = func()
        print("%s end" % func.__name__)
        return result
    return decorator

# 使用装饰器
@logging
def test01():
    return 1

# 测试用例
print(test01())
print(test01.__name__)

test01 called
test01 end
1
test01


注意`functools.wraps`用法，其目的是"`est01.__name__`输出正确的"test01"。`@logging`相当于`test01 = logging(test01)`，返回的是`decorator`函数，所以如果不加`functools.wraps`，则`test01.__name__`返回为`decorator`。

（2）装饰器传入函数参数，并正确返回结果：b

In [62]:
def logging(func):
    @functools.wraps(func)
    def decorator(*args, **kw):
        print("%s called" % func.__name__)
        result = func(*args, **kw)
        print("%s ended" % func.__name__)
        return result
    return decorator

@logging
def test02(a, b, c=1):
    print('in the func test02, a=%s, b=%s, c=%s' % (a,b,c))
    return 1

print(test02(1,2,c=3))

test02 called
in the func test02, a=1, b=2, c=3
test02 ended
1


可以 多个装饰器同时装饰一个函数

（3））将装饰器写为类的形式，即“装饰器类”。此时对于装饰器类的要求是必须是可被调用的，即必须实现类的__call__方法。

In [63]:
class Decorator(object):
    def __init__(self, func):
        self.func = func
        return
    
    def __call__(self, *args, **kw):
        print("%s called" % self.func.__name__)
        result = self.func(*args, **kw)
        print("%s ended" % self.func.__name__)
        return result
    
@Decorator
def test03(a, b, c):
    print("in function test03 a=%s b=%s c=%s" %(a,b,c))
    return 1
print(test03(1,2,3))

test03 called
in function test03 a=1 b=2 c=3
test03 ended
1


#### 其他一些装饰器实例
函数缓存：一个函数的执行结果可以被缓存在内存中，下次再次调用时，可以先查看缓存中是否存在

如果存在则直接返回缓存中的结果，否则返回函数调用结果。

这种装饰器比较适合装饰过程比较复杂或耗时的函数，比如数据库查询等。

In [ ]:
def funccache(func):
    cache = {}
    
    @functools.wraps(func)
    def _inner(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cahce[args]
    return _inner

@funcache
def test04(a, b, c):
    ### 各种耗时的计算操作
    return a+b+c

#### Python中自带的装饰器
Python中自带有三个和class相关的装饰器：`@staticmethod`、`@classmethod` 和`@property`。

（1）`@property`，可以将其理解为“将类方法转化为类属性的装饰器”

In [64]:
class People(object):
    
    def __init__(self):
        self._name = None
        self._age = None
        return
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        self._name = name
        return
    
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        assert 0<age<120
        self._age = age
        return
    
p = People()
p.name="LonelVino"
p.age = 12
print(p.name, p.age)
p.age = 120

LonelVino 12


AssertionError: 

这里定义一个People类，有两个属性name和age。

当我们声明了实例p，使用p操作name和age时，实际上是调用的name、age方法，此时会做参数检查等工作。

`@property`将name方法转化为属性，同时当对该属性进行赋值时，会自动调用`@name.sette`r将下边的name方法。

`@propert`y有`.setter`、`.getter`和`.deleter`三中装饰器，分别对应赋值、取值和删除三种操作。

（2）`@staticmethod` 将类成员方法声明为类静态方法，类静态方法没有 self 参数，可以通过类名或类实例调用。

（3）`@classmethod` 将类成员方法声明为类方法，类方法所接收的第一个参数不是self，而是cls，即当前类的具体类型。

静态方法和类方法都比较简单，一个简单的例子解释静态方法和类方法：



In [65]:
class A(object):
    var = 1
    def func(self):
        print(self.var)
        return
    
    @statemethod
    def static_func():
        print(A.var)
        return
    
    @classmethod
    def class_func(cls):
        print(cls.var)
        cls().func()
        return

NameError: name 'statemethod' is not defined